In [4]:
import requests
import pandas as pd

In [23]:
class Libraries:
    def __init__(self, api_file_path='./local/api.txt'):
        
        self.api_file_path = api_file_path
        
        self.api_key = None
        self.payload = None
        self.url = None
        self.r  = None
        
        self.load_api_key()
        
        return
    
    def load_api_key(self):
        with open(self.api_file_path, 'r') as file:
            self.api_key = file.read()
        return
    
    def create_payload(self, paramenters=None):
        self.payload = dict()
        self.payload.update({'api_key': self.api_key})
        return
    
    def get_response(self):
        self.r = requests.get(self.url, params=self.payload)
        return
    
    def get_package(self, repository='Pypi', package='requests'):
        self.url = 'https://libraries.io/api/{}/{}'.format(repository, package)
        self.get_response()
        return self.r.json()
    
lib = Libraries()
lib.get_package(repository='Pypi', package='numpy')
    

{'dependent_repos_count': 84861,
 'dependents_count': 4992,
 'deprecation_reason': None,
 'description': 'NumPy is the fundamental package for array computing with Python.',
 'forks': 4732,
 'homepage': 'https://www.numpy.org',
 'keywords': ['numpy', 'python'],
 'language': 'C',
 'latest_download_url': None,
 'latest_release_number': '1.19.1',
 'latest_release_published_at': '2020-07-21T20:54:49.000Z',
 'latest_stable_release': {'id': 30141184,
  'project_id': 61796,
  'number': '1.19.1',
  'published_at': '2020-07-21T20:54:49.000Z',
  'created_at': '2020-07-21T20:58:17.905Z',
  'updated_at': '2020-07-21T20:58:17.905Z',
  'runtime_dependencies_count': None,
  'spdx_expression': 'NOASSERTION',
  'original_license': 'BSD',
  'researched_at': None},
 'latest_stable_release_number': '1.19.1',
 'latest_stable_release_published_at': '2020-07-21T20:54:49.000Z',
 'license_normalized': True,
 'licenses': 'BSD',
 'name': 'numpy',
 'normalized_licenses': ['BSD-3-Clause'],
 'package_manager_url': 

In [8]:
r = requests.get('https://libraries.io/api/Pypi/numpy', params=payload)
data = r.json()

In [ ]:
r = requests.get('https://libraries.io/api/platforms', params=payload)

In [10]:
data.keys()

dict_keys(['dependent_repos_count', 'dependents_count', 'deprecation_reason', 'description', 'forks', 'homepage', 'keywords', 'language', 'latest_download_url', 'latest_release_number', 'latest_release_published_at', 'latest_stable_release', 'latest_stable_release_number', 'latest_stable_release_published_at', 'license_normalized', 'licenses', 'name', 'normalized_licenses', 'package_manager_url', 'platform', 'rank', 'repository_url', 'stars', 'status', 'versions'])